# Cluster Analysis, ANN, & Text Mining: Part 3 - ANN
### Author: Michael Berbach

In [ ]:
import pandas as pd
from keras import Sequential
from keras.layers import Dense
import numpy as np
from sklearn import preprocessing as prep

# loading data set from assignment 2
dfa = pd.read_csv('./Admission_Predict_Ver1.1_small_data_set_for_Linear_Regression')

